cross entropy loss　分類問題

In [1]:
#　colaboraory用: Google drive をマウントする
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# colaboratory用: フォルダを移る
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


フォルダは自分の指定のものに変更してね。

In [4]:
%cd "ctorch210701/MiniCTorch_Prototype"

/content/drive/My Drive/Colab Notebooks/ctorch210701/MiniCTorch_Prototype


In [5]:
! pip install lark-parser

     |████████████████████████████████| 229 kB 4.9 MB/s 
  Created wheel for lark-parser: filename=lark_parser-0.11.3-py2.py3-none-any.whl size=99739 sha256=4346dfa4c3cdefc54004053c2711aef257c6ac0f15a1ad65127977183b6a2aaa
  Stored in directory: /root/.cache/pip/wheels/2d/f0/65/7aea47a49de7acac0108aac16a7ee00eb996f872d978feb87e
Successfully built lark-parser


In [7]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import minictorch
import minictorch.generator as GN
import minictorch.converter as CV

ニューラルネットワークの定義

In [8]:
class Net(torch.nn.Module):
  def __init__( self, t ):
    super(Net, self).__init__()
    #self.fc1 = nn.Linear(32, 16)
    #self.fc2 = nn.Linear(16, 1)
    self.target = t
  
  def forward(self,x):
    #x = F.relu(self.fc1(x))
    #x = self.fc2(x)
    loss = nn.CrossEntropyLoss()
    output = loss(x,self.target)
    return output

In [9]:
def generate_json( json_path, input, target ):

    model = Net( target )

    model.eval()
    with torch.no_grad():
        print("[SAVE]", json_path )
        GN.generate_minictorch_file( model, input, json_path )

    return model

torch.manual_seed( 1 )

input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

project = 'cse1'
json_path = 'network/' + project +'.json'

model = generate_json( json_path, input, target )

with torch.set_grad_enabled(True):
  output = model( input )
  print(output)
  output.backward()
  print("input",input.grad)

[SAVE] network/cse1.json
tensor(1.9087, grad_fn=<NllLossBackward>)
input tensor([[ 0.0949,  0.0640, -0.2812,  0.0912,  0.0312],
        [ 0.0817,  0.0210,  0.1412, -0.2988,  0.0549],
        [ 0.0335,  0.0770, -0.2662,  0.0311,  0.1246]])


In [10]:
def convert_json( project, model, input_x, json_path, rand_flag=0 ):

    folder = "src"
    cpp_fname   = project + ".cpp"
    param_fname = project + "_param.cpp"
    cpp_path    = folder + "/" + cpp_fname
    param_path  = folder + "/" + param_fname
    make_path   = folder + "/" + "Makefile"

    # load json file
    print( "[JSON]", json_path )
    fp = open( json_path )
    obj = json.load( fp )

    # save parameter file
    code1 = CV.c_param_generator( obj, model, input_x )
    if len( code1 ) > 0:
       print( "[PARAM]", param_path )
       ofparam = open( param_path, "w" )
       ofparam.write( code1 )

    # save cpp file
    print( "[CPP]  ", cpp_path )
    code2 = CV.c_code_generator( obj, model, rand_flag )

    #ofp=open(args.path+"/"+args.output,"w")
    ofp = open( cpp_path, "w" )
    ofp.write( code2 )

    # save make file
    print( "[MAKE] ", make_path )
    make_code = CV.makefile_generator( cpp_fname )

    #makefp=open(args.path+"/"+"Makefile","w")
    makefp = open( make_path, "w" )
    makefp.write( make_code )

convert_json( project, model, input, json_path )

[JSON] network/cse1.json
[PARAM] src/cse1_param.cpp
[CPP]   src/cse1.cpp
{'name': 'input/x', 'op': 'IO Node', 'in': [], 'shape': [3, 5], 'out': [5], 'sorted_id': 0}
{'name': 'Net/4', 'op': 'prim::Constant', 'in': [], 'shape': [3], 'constant_value': [2.0, 3.0, 2.0], 'out': [5], 'sorted_id': 1}
{'name': 'Net/5', 'op': 'prim::Constant', 'in': [], 'shape': [], 'out': [5], 'sorted_id': 2}
{'name': 'Net/6', 'op': 'prim::Constant', 'in': [], 'shape': [], 'constant_value': 1.0, 'out': [5], 'sorted_id': 3}
{'name': 'Net/7', 'op': 'prim::Constant', 'in': [], 'shape': [], 'constant_value': -100.0, 'out': [5], 'sorted_id': 4}
{'name': 'Net/8', 'op': 'aten::cross_entropy_loss', 'in': [0, 1, 2, 3, 4], 'shape': [], 'out': [6], 'sorted_id': 5}
{'name': 'output/output.1', 'op': 'IO Node', 'in': [5], 'shape': [], 'out': [], 'sorted_id': 6}
[MAKE]  src/Makefile


In [12]:
!g++ -std=c++14 ./src/cse1.cpp ./src/cse1_param.cpp -I ../../ctorch/lib -lblas -o cse1

In [13]:
!./cse1

### forward computation ...
ashape35
 1.90869 
### backward computation ...
ashape3,5
input_grad{{ 0.094905,  0.063972, -0.281232,  0.09118 ,  0.031175},
 { 0.081661,  0.02103 ,  0.14123 , -0.298829,  0.054908},
 { 0.033458,  0.07704 , -0.266174,  0.031092,  0.124585}}
